### Response Figure 1

Since this manuscript's main advance regarding variation in paternal age effect over the Rahbari, et al. result is greater statistical power, more robust statistical analyses of this pattern would strengthen the paper. Figure 3 presents a commendable amount of raw data in a fairly clear way, yet the authors use only a simple ANOVA to test whether different families have different dependencies on paternal age. The supplement claims that this result cannot be an artifact of low sequencing coverage because regions covered by <12 reads are excluded from the denominator, but there still might be subtle differences in variant discovery power between e.g. regions covered by 12 reads and regions covered by 30 reads. To hedge against this, the authors can define the "callable genome" continuously (point 1 under "other questions and suggestions") or they can check whether mean read coverage appears to covary with mutation rates across individuals after filtering away the regions covered by <12 reads.

In [ ]:
library(ggplot2)
library(cowplot)

# read in second- and third-generation DNMs (now with a column for mean read depth)
gen2 = read.csv("../data/second_gen.dnms.summary.csv")
gen3 = read.csv("../data/third_gen.dnms.summary.csv")

gen2$generation = "2nd"
gen3$generation = "3rd"

# combine the second- and third-generation dataframes, and calculate
# autosomal mutation rates for all samples
combined = rbind(gen2, gen3)
combined$autosomal_mutation_rate = combined$autosomal_dnms / (combined$autosomal_callable_fraction * 2)

# generate the figure
p <- ggplot(combined, aes(x=autosomal_mutation_rate, y=mean_depth)) +
        facet_wrap(~generation) +
        geom_smooth(method="lm", aes(col=generation)) +
        geom_point(aes(fill=generation), pch=21, col="white", size=2) +
        xlab("Autosomal mutation rate") +
        ylab("Mean autosomal read depth\n(sites >= 12 reads)") +
        theme(axis.text.x = element_text(angle=45, vjust=0.5)) +
        coord_fixed(ratio=9.5e-10)

# fit models predicting mean autosomal read depth as a function
# of the autosomal mutation rate
m_second_gen = lm(mean_depth ~ autosomal_mutation_rate, data=subset(combined, generation=="2nd"))
m_third_gen = lm(mean_depth ~ autosomal_mutation_rate, data=subset(combined, generation=="3rd"))

# test whether read depth and mutation rates are correlated
summary(m_second_gen)
summary(m_third_gen)

p

### Response Figure 2

Another concern about paternal age effects is the extent to which outlier offspring may be driving the apparent rate variation across families. If the authors were to randomly sample half of the children from each family and run the analysis again, how much is the paternal age effect rank preserved? Alternatively, how much is the family rank ordering preserved if mutations are only called from a subset of the chromosomes?

In [ ]:
library(MASS)
library(ggplot2)
library(cowplot)
library(ggridges)
library(tidyr)
library(reshape2)
library(dplyr)
library(viridis)

set.seed(12345)

# read in third-generation DNMs
gen3 = read.csv("../../dnm-analysis/ceph/manuscript/third_gen.dnms.summary.csv")

# number of subsampling experiments to perform
n_sims = 100

# empty dataframe to store ranks from subsampling experiments
sub_df = data.frame(matrix(NA, nrow=n_sims*40, ncol=2))
# empty dataframe to store original ranks
orig_df = data.frame()

fam_list = unique(gen3$family_id)

# this function accepts a dataframe with information
# about each family's slope, and sorts it in order of increasing
# slope
sort_and_index <- function(df) {
    df = df[order(df$slope),]
    df$facet_order = factor(df$family_id, levels = unique(df$family_id))
    df = df[!duplicated(df[,c('family_id')]),]
    df$order = rev(c(1:nrow(df)))
    return(df)
}

# loop over every family, fit a regression (using all of the
# third-generation samples in the family), and store the results
# in `orig_df`
for (sp in split(gen3, as.factor(gen3$family_id))) {
    m = glm(autosomal_dnms ~ dad_age, data=sp, family=poisson(link="identity"))
    s = summary(m)
    sp$slope = s$coefficients[[2]]
    sp$intercept = s$coefficients[[1]]
    orig_df = rbind(orig_df, sp)
}

# sort and rank results from the full third-generation dataset
orig_df = sort_and_index(orig_df)
orig_rank_order = rev(c(as.character(orig_df$family_id)))

# for the specified number of simulations, subsample each
# family's children, fit a regression, and store the results in
# `sub_df`
for (iter in 1:n_sims) {
    new_gen3 = data.frame()
    for (sp in split(gen3, as.factor(gen3$family_id))) {
        # sub sample the family's children (here, 75% of children)
        sub_sampled = sp[sample(1:nrow(sp), nrow(sp) * 0.75, replace=FALSE),] 
        m = glm(autosomal_dnms ~ dad_age, data=sub_sampled, family=poisson(link="identity"))
        s = summary(m)
        sub_sampled$slope = s$coefficients[[2]]
        new_gen3 = rbind(new_gen3, sub_sampled)
    }
    
    # sort and rank results
    sorted_df = sort_and_index(new_gen3)
    
    # add the results to the main `sub_df`
    for (f in 1:length(fam_list)) {
        # get the index (rank) of each family ID in `sorted_df`,
        # which contains the new rank of each family ID after subsampling
        sub_df[iter * f,2] = which(fam_list[[f]] == sorted_df$family_id)
        sub_df[iter * f,1] = as.character(fam_list[[f]])
    }
}

colnames(sub_df) = c("family_id", "rank")

# make sure `family_id` is treated as a factor
sub_df$family_id <- as.factor(sub_df$family_id)

# create a "ridges" or "joyplot" summarizing the
# distribution of ranks for each family across 
# simulations, ordered by their original ranks
p <- ggplot(sub_df, aes(x = rank, y = family_id, fill=..x..)) + 
     geom_density_ridges_gradient(scale = 3, rel_min_height = 0.01, quantile_lines = TRUE, quantiles = 2) +
     scale_fill_gradient(name = "Rank", low="dodgerblue", high="firebrick") +
     scale_y_discrete(limits=orig_rank_order) +
     xlab("Distribution of family ranks\nfollowing 100 sampling trials") +
     ylab("Family ID (original rankings)") +
     theme(axis.text.x = element_text(size=12)) +
     theme(axis.text.y = element_text(size=10))

p

### Response Figure 3

A factor regarding paternal age effects that only mentioned briefly alluded to late in the paper are the differences in the intercept between families (Figure 3c). Do either the intercept or slope vary with the number of F2 children? Is there any (anti-)correlation between slope and intercept? It would seem odd if the intercept strongly impacts the slope since a low per-year rate probably should not correlate with a high initial rate at younger ages.

#### Is there anti-correlation between slopes and intercepts in CEPH families?

In [ ]:
# plot the correlation between family size and intercept
p <- ggplot(orig_df, aes(x=slope, y=intercept)) +
        geom_smooth(method='lm', color='firebrick', alpha=0.25) +
        geom_point(pch=21, fill="black", col="white", size=3, lwd=0.5) +
        ylab('Initial mutation count (intercept) in family') +
        xlab('Slope in family') +
        theme(axis.text.x=element_text(size=16)) +
        theme(axis.text.y=element_text(size=16)) +
        theme(text=element_text(size=16))

m = lm(intercept ~ slope, data=orig_df)
summary(m)

p

#### Anti-correlation between slopes and intercepts is expected for randomly distributed DNM counts...

In [ ]:
library(MASS)
library(ggplot2)
library(cowplot)

# set seed so that example is reproducible
set.seed(1234)

gen3 = read.csv("../data/third_gen.dnms.summary.csv")

library(dplyr)

df1 = gen3[,c("dad_age","family_id")]
FUN <- function(x) rpois(lambda=x * 1.72 + 15, n=1)

# randomly assign DNM counts to each third-generation sample based
# on their paternal age at birth
df1$dnms = lapply(df1$dad_age, FUN)
df1$dnms = as.numeric(df1$dnms)

colnames(df1) = c('age', 'fam_id', 'dnms')

In [ ]:
# get the slopes for each family and add
# to a new dataframe
new_df1 = data.frame()
for (sp in split(df1, df1$fam_id)) {
    m = glm(dnms ~ age, data=sp, family=poisson(link="identity"))
    s = summary(m)
    sp$slope = s$coefficients[[2]]
    sp$intercept = s$coefficients[[1]]
    new_df1 = rbind(new_df1, sp)
}

# get rid of duplicates (i.e., samples from the same
# family)
sort_and_index <- function(df) {
    df = df[order(df$slope),]
    df$facet_order = factor(df$fam_id, levels = unique(df$fam_id))
    df = df[!duplicated(df[,c('fam_id')]),]
    df$order = rev(c(1:nrow(df)))
    return(df)
}
    
new_df1 = sort_and_index(new_df1)

# plot slopes and intercepts
p <- ggplot(new_df1, aes(x=slope, y=intercept)) +
        geom_smooth(method='lm', color='dodgerblue', alpha=0.25) +
        geom_point(pch=21, fill="black", col="white", size=3, lwd=0.5) +
        ylab('Initial mutation count (intercept) in family') +
        xlab('Slope in family') +
        theme(axis.text.x=element_text(size=16)) +
        theme(axis.text.y=element_text(size=16)) +
        theme(text=element_text(size=16))
p

print(cor.test(new_df1$slope, new_df1$intercept))

#### ...but inter-family variability is not expected for randomly distributed DNM counts

In [ ]:
m = glm(dnms ~ age * fam_id, data=df1, family=poisson(link="identity"))
anova(m, test="Chisq")

#### Is family size correlated with the slope or intercept for a family?

In [ ]:
library(MASS)
library(ggplot2)
library(cowplot)

# read in third-generation DNMs
gen3 = read.csv("../../dnm-analysis/ceph/manuscript/third_gen.dnms.summary.csv")

orig_df = data.frame()

# this function accepts a dataframe with information
# about each family's slope, and sorts it in order of increasing
# slope
sort_and_index <- function(df) {
    df = df[order(df$slope),]
    df$facet_order = factor(df$family_id, levels = unique(df$family_id))
    df = df[!duplicated(df[,c('family_id')]),]
    df$order = rev(c(1:nrow(df)))
    return(df)
}

# loop over every family, fit a regression, and store the results
# in `orig_df`
for (sp in split(gen3, as.factor(gen3$family_id))) {
    m = glm(autosomal_dnms ~ dad_age, data=sp, family=poisson(link="identity"))
    s = summary(m)
    sp$slope = s$coefficients[[2]]
    sp$intercept = s$coefficients[[1]]
    orig_df = rbind(orig_df, sp)
}

# sort the rank results from the full third-generation dataset
orig_df = sort_and_index(orig_df)

# plot the correlation between family size and slope
p1 <- ggplot(orig_df, aes(x=n_sibs, y=slope)) +
        geom_smooth(method='lm', color='firebrick', alpha=0.25) +
        geom_point(pch=21, fill="black", col="white", size=4, lwd=0.5) +
        ylab('Paternal age effect (slope) in family') +
        xlab('Number of siblings in family') +
        theme(axis.text.x=element_text(size=16)) +
        theme(axis.text.y=element_text(size=16)) +
        theme(text=element_text(size=16))


m = lm(slope ~ n_sibs, data=orig_df)
summary(m)

# plot the correlation between family size and intercept
p2 <- ggplot(orig_df, aes(x=n_sibs, y=intercept)) +
        geom_smooth(method='lm', color='firebrick', alpha=0.25) +
        geom_point(pch=21, fill="black", col="white", size=4, lwd=0.5) +
        ylab('Initial mutation count (intercept) in family') +
        xlab('Number of siblings in family') +
        theme(axis.text.x=element_text(size=16)) +
        theme(axis.text.y=element_text(size=16)) +
        theme(text=element_text(size=16))


m = lm(intercept ~ n_sibs, data=orig_df)
summary(m)

p1
p2

## Other questions and suggestions 

### Suggestion #5. 

Other things to explore further related to parental age effects are: how do the conclusions change and/or can you detect similar variability in maternal age when analyzing phased DNMs? This may be underpowered, but for those families that share grandparents, if two brothers are in the F1 generation, do their paternal age effects differ?

In [ ]:
gen3 = read.csv('../data/third_gen.dnms.summary.csv')

# fit model with only paternally-phased counts
m = glm(dad_dnms_auto ~ dad_age * family_id, data=gen3, family=poisson(link="identity"))

anova(m, test="Chisq")

# fit model with only maternally-phased counts.
# need to add a pseudo-count of 1 to maternal DNMs first.
gen3$mom_dnms_auto = gen3$mom_dnms_auto + 1

m = glm(mom_dnms_auto ~ mom_age * family_id, data=gen3, family=poisson(link="identity"))

anova(m, test="Chisq")

# identify difference in paternal age effects between pairs of brothers

# first, for family 24
gen3_fam24 = subset(gen3, family_id %in% c("24_C", "24_D"))

m_exp = glm(autosomal_dnms ~ dad_age * family_id, data=gen3_fam24, family=poisson(link="identity"))
m_null = glm(autosomal_dnms ~ dad_age, data=gen3_fam24, family=poisson(link="identity"))
anova(m_null, m_exp, test="Chisq")


# next, for family 19
gen3_fam19 = subset(gen3, family_id %in% c("19_A", "19_B"))

m_exp = glm(autosomal_dnms ~ dad_age * family_id, data=gen3_fam19, family=poisson(link="identity"))
m_null = glm(autosomal_dnms ~ dad_age, data=gen3_fam19, family=poisson(link="identity"))
anova(m_null, m_exp, test="Chisq")